#### Загрузка и изучение данных

In [1]:
pip install scikit-learn==1.3.2 -q 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install shap -q 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xgboost lightgbm catboost -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install seaborn -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import re
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from scipy.stats import shapiro
from pandas.api.types import is_integer_dtype
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
import shap
from typing import Optional, List

import warnings
warnings.filterwarnings('ignore')

In [6]:
RANDOM_STATE = 42
TEST_SIZE = 0.25
TARGET_COL = "heart_attack_risk_binary"

In [7]:
heart_train = pd.read_csv('C:/Users/malen/OneDrive/Desktop/data_since/мастерская 1/data/raw/heart_train.csv', index_col='id')
heart_train.head()

Unnamed: 0       Age  Cholesterol  Heart rate  Diabetes  Family History  \
id                                                                              
2664           0  0.359551     0.732143    0.074244       1.0             1.0   
9287           1  0.202247     0.325000    0.047663       1.0             1.0   
5379           2  0.606742     0.860714    0.055912       1.0             0.0   
8222           3  0.730337     0.007143    0.053162       0.0             0.0   
4047           4  0.775281     0.757143    0.021998       0.0             0.0   

      Smoking  Obesity  Alcohol Consumption  Exercise Hours Per Week  ...  \
id                                                                    ...   
2664      1.0      1.0                  1.0                 0.535505  ...   
9287      0.0      0.0                  1.0                 0.068690  ...   
5379      1.0      1.0                  1.0                 0.944001  ...   
8222      1.0      0.0                  1.0                 0.697023  ...   
4047      1.0      0.0                  1.0                 0.412878  ...   

      Triglycerides  Physical Activity Days Per Week  Sleep Hours Per Day  \
id                                                                          
2664       0.979221                              3.0             0.333333   
9287       0.515584                              3.0             0.833333   
5379       0.012987                              2.0             1.000000   
8222       0.131169                              0.0             0.333333   
4047       0.070130                              5.0             1.000000   

      Heart Attack Risk (Binary)  Blood sugar     CK-MB  Troponin  Gender  \
id                                                                          
2664                         0.0     0.227018  0.048229  0.036512    Male   
9287                         0.0     0.150198  0.017616  0.000194  Female   
5379                         0.0     0.227018  0.048229  0.036512  Female   
8222                         1.0     0.227018  0.048229  0.036512  Female   
4047                         1.0     0.227018  0.048229  0.036512    Male   

      Systolic blood pressure  Diastolic blood pressure  
id                                                       
2664                 0.212903                  0.709302  
9287                 0.412903                  0.569767  
5379                 0.238710                  0.220930  
8222                 0.348387                  0.267442  
4047                 0.619355                  0.441860  

[5 rows x 27 columns]

In [8]:
heart_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8685 entries, 2664 to 7270
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       8685 non-null   int64  
 1   Age                              8685 non-null   float64
 2   Cholesterol                      8685 non-null   float64
 3   Heart rate                       8685 non-null   float64
 4   Diabetes                         8442 non-null   float64
 5   Family History                   8442 non-null   float64
 6   Smoking                          8442 non-null   float64
 7   Obesity                          8442 non-null   float64
 8   Alcohol Consumption              8442 non-null   float64
 9   Exercise Hours Per Week          8685 non-null   float64
 10  Diet                             8685 non-null   int64  
 11  Previous Heart Problems          8442 non-null   float64
 12  Medication Use        

heart_train состоит из 8685 записей и 27 столбцов, 243 пропуска в отдельных столбцах. Требуется изменение типа данных object.

In [9]:
heart_test = pd.read_csv('C:/Users/malen/OneDrive/Desktop/data_since/мастерская 1/data/raw/heart_test.csv', index_col='id')
heart_test.head()

Unnamed: 0       Age  Cholesterol  Heart rate  Diabetes  Family History  \
id                                                                              
7746           0  0.494382     0.264286    0.062328       0.0             1.0   
4202           1  0.224719     0.953571    0.082493       1.0             0.0   
6632           2  0.629213     0.092857    0.064161       0.0             1.0   
4639           3  0.460674     0.567857    0.055912       1.0             1.0   
4825           4  0.719101     0.485714    0.022915       1.0             0.0   

      Smoking  Obesity  Alcohol Consumption  Exercise Hours Per Week  ...  \
id                                                                    ...   
7746      1.0      1.0                  1.0                 0.361618  ...   
4202      0.0      1.0                  0.0                 0.996483  ...   
6632      1.0      1.0                  0.0                 0.995561  ...   
4639      1.0      1.0                  1.0                 0.437277  ...   
4825      1.0      0.0                  1.0                 0.514920  ...   

           BMI  Triglycerides  Physical Activity Days Per Week  \
id                                                               
7746  0.283490       0.306494                              1.0   
4202  0.467036       0.087013                              0.0   
6632  0.409366       0.205195                              7.0   
4639  0.910261       0.163636                              0.0   
4825  0.758924       0.580519                              5.0   

      Sleep Hours Per Day  Blood sugar     CK-MB  Troponin  Gender  \
id                                                                   
7746             0.333333     0.227018  0.048229  0.036512    Male   
4202             0.166667     0.227018  0.048229  0.036512  Female   
6632             1.000000     0.102767  0.002666  0.088455    Male   
4639             0.666667     0.203557  0.056390  0.271774  Female   
4825             0.000000     0.227018  0.048229  0.036512    Male   

      Systolic blood pressure  Diastolic blood pressure  
id                                                       
7746                 0.283871                  0.372093  
4202                 0.703226                  0.441860  
6632                 0.458065                  0.779070  
4639                 0.741935                  0.255814  
4825                 0.412903                  0.395349  

[5 rows x 26 columns]

In [10]:
heart_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966 entries, 7746 to 6521
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       966 non-null    int64  
 1   Age                              966 non-null    float64
 2   Cholesterol                      966 non-null    float64
 3   Heart rate                       966 non-null    float64
 4   Diabetes                         935 non-null    float64
 5   Family History                   935 non-null    float64
 6   Smoking                          935 non-null    float64
 7   Obesity                          935 non-null    float64
 8   Alcohol Consumption              935 non-null    float64
 9   Exercise Hours Per Week          966 non-null    float64
 10  Diet                             966 non-null    int64  
 11  Previous Heart Problems          935 non-null    float64
 12  Medication Use         

heart_test состоит из 8685 записей и 26 столбцов, 31 пропуск в отдельных столбцах. Требуется изменение типа данных object.

In [11]:
heart_train['Unnamed: 0'].value_counts()

Unnamed: 0
0       1
5785    1
5799    1
5798    1
5797    1
       ..
2890    1
2889    1
2888    1
2887    1
8684    1
Name: count, Length: 8685, dtype: int64

In [12]:
heart_test['Unnamed: 0'].value_counts()

Unnamed: 0
0      1
604    1
636    1
637    1
638    1
      ..
326    1
327    1
328    1
329    1
965    1
Name: count, Length: 966, dtype: int64

данный столбец использовался для нумерации строк в таблице, поэтому он нам не нужен.

In [13]:
heart_train = heart_train.drop('Unnamed: 0', axis=1)
heart_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8685 entries, 2664 to 7270
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8685 non-null   float64
 1   Cholesterol                      8685 non-null   float64
 2   Heart rate                       8685 non-null   float64
 3   Diabetes                         8442 non-null   float64
 4   Family History                   8442 non-null   float64
 5   Smoking                          8442 non-null   float64
 6   Obesity                          8442 non-null   float64
 7   Alcohol Consumption              8442 non-null   float64
 8   Exercise Hours Per Week          8685 non-null   float64
 9   Diet                             8685 non-null   int64  
 10  Previous Heart Problems          8442 non-null   float64
 11  Medication Use                   8442 non-null   float64
 12  Stress Level          

In [14]:
heart_test = heart_test.drop('Unnamed: 0', axis=1)
heart_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966 entries, 7746 to 6521
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              966 non-null    float64
 1   Cholesterol                      966 non-null    float64
 2   Heart rate                       966 non-null    float64
 3   Diabetes                         935 non-null    float64
 4   Family History                   935 non-null    float64
 5   Smoking                          935 non-null    float64
 6   Obesity                          935 non-null    float64
 7   Alcohol Consumption              935 non-null    float64
 8   Exercise Hours Per Week          966 non-null    float64
 9   Diet                             966 non-null    int64  
 10  Previous Heart Problems          935 non-null    float64
 11  Medication Use                   935 non-null    float64
 12  Stress Level           